In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Phase 1
## **Step 3 (1分):** 
在接触实际数据之前，我们对项目有了初步的认识，那么我们来看看实际的数据吧！请使用pandas的read_csv函数导入项目数据（train和test）

In [ ]:
# 代码写在下面

import pandas as pd

train_data = pd.read_csv('/kaggle/input/porto-seguro-safe-driver-prediction/train.csv')
test_data = pd.read_csv('/kaggle/input/porto-seguro-safe-driver-prediction/test.csv')

## **Step 4 (1分)**: 
尝试用 info(), head(), tail() 函数检查训练数据和测试数据

In [ ]:
# print(train_data.info())
# print(train_data.head(1))
# print(train_data.tail(1))

# print(test_data.info())
# print(test_data.head(1))
# print(test_data.tail(1))

> **以下是比赛数据的基本描述：**

由于对于客户安全性的考虑，特征名称没用明确给出。

属于相似分类的特征在特征名称中被标记为（例如，ind、reg、car、calc）。
特征名称包括表示二元特征的后缀 bin（0 or 1） 和表示分类特征的 cat。
特征没有这些名称是连续的或有序的。
-1 的值表示该特征在观察中缺失。
target 列表示是否为该保单持有人提出索赔。
让我们快速用代码查看数据的基本情况吧！

In [ ]:
# TODO: info() 检查特征名字、非空值以及变量类型
# print(train_data.info())
# print(test_data.info())

# TODO: head() 查看前10行
# print(train_data.head(10))
# print(test_data.head(10))


# TODO: tail() 查看后7行
# print(train_data.tail(7))
# print(test_data.tail(7))

## 👊**附加挑战（2分）**

因为我们常常要上面的方法去检查数据，有一个基本的理解，那么我们想一下子看到数据的前五行和后五行，合并在一起。

请编写一个函数，名为check_data，输入为dataframe，输出为一个新dataframe，满足一下条件：

一共10行，列数为输入dataframe的列数
前5行是输入dataframe的前五行，后5行是输入dataframe的后五行

In [ ]:
def check_data(data):
    
    new_dataframe = pd.concat([data.head(5),data.tail(5)],axis=0)
    
    return new_dataframe

check_data(train_data)

# **Phase 2**


## **Step 1 (1分):** 
我们应该从那几个方面梳理高维度数据？写出你的想法，并与组员讨论，给出至少3个维度。

**Ans:**
疑问：先理解什么是高维度数据？

梳理高纬度数据想法：<br >
1.求解各个维度数据之间的相关性（corr）去除相关性不高的数据(variance filter) <br >
2.查看某个维度中缺失值数量有多少(missing value ratio)<br >
3.降低维维度进行分析，使用LDA/PCA<br >
4.分析各维度中影响因子的大小or占比(factor analysis)


## **Step 2 (2分)**:  
构建元数据结构表。

In [ ]:
# train的长度
train_length = len(train_data)

# 变量名称
key_of_data = train_data.columns

# 数据角色
character_of_data_list = []
# 变量类型
variable_of_data_list = []
# 数据类型
type_of_data = train_data.dtypes
# 是否保留
flag_list = []

# 唯一值数量
number_of_ones = []
# 缺失值数量
number_of_missing_value = []

# 利用for循环遍历变量名称
for i in range(len(key_of_data)):
    if key_of_data[i] == 'id':
        character_of_data_list.append('序列号(不属于特征)')
        flag_list.append('False')
    elif key_of_data[i] == 'target':
        character_of_data_list.append('目标标签(是否发起索赔)')
        flag_list.append('True')
    else:
        character_of_data_list.append('特征')
        flag_list.append('True')

    
    if 'bin' in key_of_data[i] or key_of_data[i] == 'Target':
        variable_of_data_list.append('binary')
    elif 'cat' in key_of_data[i] or key_of_data[i] == 'id':
        variable_of_data_list.append('nominal')
    elif'float' in str(type_of_data[i]):
        variable_of_data_list.append('interval')
    elif 'int' in str(type_of_data[i]):
        variable_of_data_list.append('ordinal')
    
    if key_of_data[i] == 'id': 
        number_of_ones.append('不统计')
    else:
        temp = set(train_data[key_of_data[i]])
        if -1 in temp:
            print("特征为 {} 时有缺失值".format(key_of_data[i]))
            number_of_ones.append(len(temp) - 1)
        else:
            number_of_ones.append(len(temp))
    
    
    t = len(train_data[train_data[key_of_data[i]] == -1])  # 当‘值’为-1时为缺失值。
    t_percentage = round(t / train_length * 100,2)
    number_of_missing_value.append(str(t) + '(' + str(t_percentage) + '%' + ')')
    
data = {
    '数据角色':character_of_data_list,
    '变量类型':variable_of_data_list,
    '数据类型':type_of_data,
    '是否保留':flag_list,
    '唯一值数量':number_of_ones,
    '缺失值数量':number_of_missing_value
}

# print(type_of_data[1])
meta = pd.DataFrame(data, index=key_of_data)

meta

## Step 3 (3分):
请基于meta，找出各个数据角色、变量类型各有多少个特征，目标形成下图的dataframe！

In [ ]:
meta_new = meta.groupby(['数据角色','变量类型'])
t1 = meta_new.get_group(('特征', 'nominal'))
print(len(t1))

# 总数
total = []
# 变量类型
type_of_variable = []
# 数据角色
type_of_data = []

# 循环得到总数
for every in list(meta_new):
    
    total.append(len(meta_new.get_group(every[0]))) # 总数
    type_of_data.append(every[0][0]) # 数据角色
    type_of_variable.append(every[0][1]) # 变量类型
                 
df = {
    '数据角色':type_of_data,
    '变量类型':type_of_variable,
    '总数':total
}

print(type_of_data)
print(type_of_variable)

data = pd.DataFrame(df)

data

In [ ]:
meta

In [ ]:
# 先拿到连续型变量list
interval_list = [x for x in meta[meta['变量类型'] == 'interval'].index]

# 查看连续型变量的缺失值情况
for every in interval_list:
    print(every, len(train_data[train_data[every] == -1]))

# 查看train_data中，连续型变量(interval)的describe()
train_data[interval_list].describe()

## **Step 4 (3分):** 
请基于meta，对连续型变量做统计描述型分析，使用describe函数！ - 请问连续型变量里面存在缺失值的有哪些？ - 请问连续型变量里面标准差最小的变量名称是什么？ - 请问连续型变量里面最大值大于4的变量名称是什么？

**Ans:**
连续型变量？interval？
连续型变量里面存在缺失值的有哪些？

缺失值有：**ps_reg_03   ps_car_14  ps_car_12**	

连续型变量里面标准差最小的变量名称是什么？<br />
ps_car_12


请问连续型变量里面最大值大于4的变量名称是什么？
<br />
ps_reg_03

## **Step 5 (2分):** 
基于上一问，简要分析 calc 变量是个什么类型的变量，这些 calc 变量有什么相似点么？（开放式题目，合理即可给分）

**Ans:**
calc变量是interval。 也可以是ratio variable。


相似点？ <br >数据分布近似于正态分布。<br > 平均值、标准差很相似。<br > 且不同的calc变量中数据分布也非常相似。

## **Step 6（附加） (2分)**: 
根据下列代码和对应的结果，你有什么发现么？（开放式题目，合理即可给分）

**Ans:**

发现：<br >
大多数的预测结果都为0. <br >
标准差很小。<br >
目标值是imbalanced data


count    595212.000000 <br/>
mean          0.036448 <br/>
std           0.187401 <br/>
min           0.000000 <br/>
25%           0.000000 <br/>
50%           0.000000 <br/>
75%           0.000000 <br/>
max           1.000000 <br/>
Name: target, dtype: float64